#### PPGIA - Unifor - Prof. Daniel - Processamento e analise de imagens digitais - Trabalho03 - Realce


#### Importação de Bibliotecas

In [1]:
import numpy as np
import pandas as pd
import cv2 
print(cv2.__version__)
#Path Google Colab para CV2
from google.colab.patches import cv2_imshow
from google.colab import files
import dlib
from skimage import io
from PIL import Image 
%matplotlib inline
#A linha acima é necessária para mostrar gráficos Matplotlib dentro de um Notebook Jupyter
from matplotlib import pyplot as plt
#import matplotlib.pylab as plt
import sys
import ipywidgets as widgets
from IPython.display import display
import scipy
from scipy import interpolate
#import face_recognition

4.1.2


#### Upload do arquivo

In [2]:
#Upload do arquivo de imagens que será utilizado no exemplo (lena.png)
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving paisagem.jpg to paisagem.jpg
User uploaded file "paisagem.jpg" with length 32434 bytes


#### Funções

In [3]:
#Funcao para Exibir Imagens com matplotlib e cv2
def exibir_imgplt(tmpimg):
  #Importar a Imagem
  image = cv2.imread(tmpimg)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,None,fx=0.25,fy=0.25,interpolation=cv2.INTER_AREA)
  #Mostra a imagem com matplotlib
  plt.imshow(image)
  plt.show()
  return

#### Exibir a Imagem

In [4]:
#Botão para exibir a imagem que foi realizado Upload
button = widgets.Button(description="Exibir Img CV2PLT")
output = widgets.Output()

def on_button_clicked_exibir_imagem(b):
  with output:
    img = cv2.cvtColor(cv2.imread(next(iter(uploaded))), cv2.COLOR_BGR2RGB)
    print("Exibindo Imagens:")
    print("Resolução: ", img.shape[0: 2])
    #print(img) # Imprimindo a imagem como NumpyArray
    exibir_imgplt(next(iter(uploaded)))
button.on_click(on_button_clicked_exibir_imagem)
display(button, output)

Button(description='Exibir Img CV2PLT', style=ButtonStyle())

Output()

#### Exibir a Imagem em PB

In [5]:
#Funcao para Exibir Imagens em PB com matplotlib e cv2
def exibir_imgpltpb(tmpimg):
  #Importar a Imagem
  image = cv2.imread(tmpimg)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image,None,fx=0.25,fy=0.25,interpolation=cv2.INTER_AREA)
  #Mostra a imagem com matplotlib
  #plt.imshow(image)
  plt.imshow(image, cmap="gray")
  plt.show()
  return

In [6]:
#Botão para exibir a imagem que foi realizado Upload em PB
button1 = widgets.Button(description="Exibir Img CV2PLTPB")
output1 = widgets.Output()

def on_button_clicked_exibir_imagempb(b):
  with output1:
    img = cv2.cvtColor(cv2.imread(next(iter(uploaded))), cv2.COLOR_BGR2RGB)
    print("Exibindo Imagens:")
    print("Resolução: ", img.shape[0: 2])
    #print(img) # Imprimindo a imagem como NumpyArray
    exibir_imgpltpb(next(iter(uploaded)))
button1.on_click(on_button_clicked_exibir_imagempb)
display(button1, output1)

Button(description='Exibir Img CV2PLTPB', style=ButtonStyle())

Output()

#### Exibir o Histograma

In [7]:
def histograma(image):

  #Função calcHist para calcular o hisograma da imagem
  h = cv2.calcHist([image], [0], None, [256], [0, 256])
  plt.figure()
  plt.title("Histograma P&B")
  plt.xlabel("Intensidade")
  plt.ylabel("Qtde de Pixels")
  plt.plot(h)
  plt.xlim([0, 256])
  plt.show()
  cv2.waitKey(0)

  return 

def histogramacolor(image):

  #Função calcHist para calcular o hisograma da imagem separando os canais de cor
  color = ('b','g','r')
  for channel,col in enumerate(color):
      histr = cv2.calcHist([image],[channel],None,[256],[0,256])
      plt.plot(histr,color = col)
      plt.xlim([0,256])
  plt.title('Histograma da Figura dos canais de cor')
  plt.show()

  return 

In [8]:
#Botão para exibir o histograma
button2 = widgets.Button(description="Exibir Histograma")
output2 = widgets.Output()

def on_button_clicked_exibir_histograma(b):
  with output2:
    img = cv2.cvtColor(cv2.imread(next(iter(uploaded))), cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(cv2.imread(next(iter(uploaded))), cv2.COLOR_BGR2RGB)
    print("Exibindo Histograma:")
    #Apresenta o histograma
    histograma(img)
    #Apresenta o histograma colorido
    histogramacolor(img2)

button2.on_click(on_button_clicked_exibir_histograma)
display(button2, output2)

Button(description='Exibir Histograma', style=ButtonStyle())

Output()

#### Distribuição Acumulada


In [18]:
def histograma_cdf(img):

  # Calcule o histograma
  hist, bins = np.histogram(img.flatten(), 256, [0, 256])

  # Calcule a função de distribuição cumulativa do histograma
  cdf = hist.cumsum()

  # Equalize a imagem
  cdfMasked = np.ma.masked_equal(cdf, 0)
  cdfMasked = (cdfMasked - cdfMasked.min()) * 255 / (cdfMasked.max() - cdfMasked.min())
  equalizeFunction = np.ma.filled(cdfMasked, 0).astype('uint8')
  equalizedImg = equalizeFunction[img]

  # Calcule o histograma e o cdf da imagem equalizada
  equalizedHist, bins = np.histogram(equalizedImg.flatten(), 256, [0, 256])
  equalizedCdf = equalizedHist.cumsum()

  # Exibir os resultados
  plt.subplot(221)
  plt.title('Original')
  plt.imshow(img, 'gray')
  plt.xticks([])
  plt.yticks([])

  plt.subplot(222)
  plt.hist(img.flatten(), 256, [0, 256], color='r')
  plt.plot(cdf * hist.max() / cdf.max(), color='b')
  plt.xlim([0, 256])
  plt.legend(('CDF', 'Histograma'), loc='upper left')

  plt.subplot(223)
  plt.title('Equalizado')
  plt.imshow(equalizedImg, 'gray')
  plt.xticks([])
  plt.yticks([])

  plt.subplot(224)
  plt.hist(equalizedImg.flatten(), 256, [0, 256], color='r')
  plt.plot(equalizedCdf * equalizedHist.max() / equalizedCdf.max(), color='b')
  plt.xlim([0, 256])
  plt.legend(('CDF', 'Histograma'), loc='upper left')

  plt.show()

  return

In [19]:
#Botão para exibir o histograma
button3 = widgets.Button(description="Exibir HistogramaCDF")
output3 = widgets.Output()

def on_button_clicked_exibir_histograma_cdf(b):
  with output3:
    img = cv2.cvtColor(cv2.imread(next(iter(uploaded))), cv2.COLOR_BGR2GRAY)
    #img2 = cv2.cvtColor(cv2.imread(next(iter(uploaded))), cv2.COLOR_BGR2RGB)
    print("Exibindo Histograma CDF:")
    #Apresenta o histograma
    histograma_cdf(img)

button3.on_click(on_button_clicked_exibir_histograma_cdf)
display(button3, output3)

Button(description='Exibir HistogramaCDF', style=ButtonStyle())

Output()

#### Equalização de Histograma

In [13]:
def histogram_equalization(img):

  plt.imshow(img, cmap="gray")
  plt.show()
  h_eq = cv2.equalizeHist(img)
  plt.figure()
  plt.title("Histograma Equalizado")
  plt.xlabel("Intensidade")
  plt.ylabel("Qtde de Pixels")
  plt.hist(h_eq.ravel(), 256, [0,256])
  plt.xlim([0, 256])
  plt.show()
  plt.imshow(h_eq, cmap="gray")
  plt.show()
  plt.figure()
  plt.title("Histograma Original")
  plt.xlabel("Intensidade")
  plt.ylabel("Qtde de Pixels")
  plt.hist(img.ravel(), 256, [0,256])
  plt.xlim([0, 256])
  plt.show()
  cv2.waitKey(0)

  return

In [14]:
#Botão para exibir a equalização
button4 = widgets.Button(description="Exibir Histograma Equalizado")
output4 = widgets.Output()

def on_button_clicked_exibir_histogramaequalizado(b):
  with output4:
    img = cv2.cvtColor(cv2.imread(next(iter(uploaded))), cv2.COLOR_BGR2GRAY)
    print("Exibindo Histograma:")
    #Apresenta o histograma
    histogram_equalization(img)

button4.on_click(on_button_clicked_exibir_histogramaequalizado)
display(button4, output4)

Button(description='Exibir Histograma Equalizado', style=ButtonStyle())

Output()